#  Музыка больших городов

О жизни в Москве и Санкт-Петербурге придумано много мифов, таких как:

•	Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;

•	Петербург — город своеобразной культуры, непохожий на Москву.

Но для принятия рациональных решений бизнесу необходимо отличать стереотипы от действительности.
Сравним музыкальное поведение пользователей двух столиц на реальных данных Яндекс.Музыки. 


**Исходные данные:** данные пользователей Яндекс.Музыки для Москвы и Санкт-Петербурга

**Цель исследования:** сравнение поведения пользователей двух столиц

**Задачи исследования:** проверить следующие гипотезы:

1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.


## 1 Обзор данных

In [1]:
# импорт библиотек
import pandas as pd

Составим первое представление об имеющихся данных Яндекс.Музыки. Выведем на экран первые десять строк таблицы:

In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('/Users/olgakozlova/Desktop/Практикум/В портфолио/1 Музыка больших городов - не готово/yandex_music_project.csv')

# вывод первых 10-и строк
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


Изучим общую информацию о датасете:

In [3]:
# получение общей информации о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны следующие нарушения стиля:
1. Строчные буквы сочетаются с прописными;
2. Встречаются пробелы;
3. Присутствие CamelCase.

Количество значений в столбцах различается, что говорит нам о наличии в данных пропусков.


### 1.1 Выводы

Каждая строка датасета содержит подробную информация о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что данных для проверки гипотез достаточно. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Необходимо устранить проблемы в данных: избавиться от пропущенных значений и привести данные к змеиному регистру.

## 2 Предобработка данных

Исправим стиль в заголовках столбцов, исключим пропуски, проверим данные на дубликаты.

### 2.1 Стиль заголовков

In [4]:
# перечень названий столбцов
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Приведем названия в соответствие с хорошим стилем:
* воспользуемся snake_case;
* все символы сделаем строчными;
* устраним пробелы.

In [5]:
# переименование столбцов
df = df.rename(columns={'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day': 'day'})

# проверка результатов - список названий столбцов
df.columns.to_list()

['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day']

### 2.2 Пропуски значений
Сначала посчитаем, сколько пропущенных значений содержат данные.

In [6]:
# подсчёт пропусков
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Не все пропущенные значения влияют на исследование. Пропуски в столцах `track` и `artist` не важны, достаточно заменить их явными обозначениями.

Но пропуски в столбце `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. В учебном проекте нет возможности установить причину пропусков и восстановить данные, поэтому остается:

* так же заполнить пропуски явными обозначениями;
* оценить, насколько они повредят расчётам. 

In [7]:
# замена пропущенных значений на 'unknown'
df = df.fillna('unknown')

Убедимся, что в таблице не осталось пропусков.

In [8]:
# подсчёт пропусков
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### 2.3 Дубликаты
Посчитаем и удалим явные дубликаты в таблице.

In [9]:
# подсчёт явных дубликатов
df.duplicated().sum()

3826

In [10]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True)

Убедимся, что полностью избавились от дубликатов:

In [11]:
# проверка на отсутствие явных дубликатов
df.duplicated().sum()

0

Название одного и того же жанра может быть записано немного по-разному. Такие ошибки повлияют на результат исследования. Избавимся от неявных дубликатов в колонке `genre`. Выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке.

In [12]:
# просмотр уникальных названий жанров
df = df.sort_values(by='genre')
df['genre'].unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Посмотрим на список, сразу можно увидеть следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

Напишем и вызовем функцию, передав ей такие аргументы, чтобы она устранила неявные дубликаты: вместо hip, hop и hip-hop в таблице будет значение hiphop.

In [13]:
# функция для замены неявных дубликатов
def replace_wrong_genres(wrong_genres, correct_genre):
    for genre in wrong_genres:
        df['genre'] = df['genre'].replace(genre, correct_genre)
wrong_genres = ['hip', 'hip-hop', 'hop']
correct_genre = 'hiphop'

In [14]:
# вызов функции
replace_wrong_genres(wrong_genres, correct_genre)

Проверим, что заменили неправильные названия:

In [15]:
# Проверка на неявные дубликаты
display(df['genre'].unique())

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

### 2.4 Выводы

В ходе предобработки данных:

- Исправили нарушения в стиле заголовков;
- Пропущенные значения заменили на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`;
- Избавились от явных и неявных дубликатов.


Перейдем к проверке гипотез. 

## 3 Проверка гипотез

### 3.1 Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение.

Подсчитаем количество прослушиваний по городам:

In [16]:
# Подсчёт прослушиваний в каждом городе
df.groupby('city')['genre'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: genre, dtype: int64

В Москве прослушиваний в 2.3 раза больше, чем в Петербурге. Это не означает, что московские пользователи чаще слушают музыку, просто пользователей в Москве больше.

Подсчитаем количество прослушиваний по дням недели:

In [17]:
# Подсчёт прослушиваний по дням недели
df.groupby('day')['genre'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: genre, dtype: int64

В данных имеется информация только по трем дням недели: понедельник, среда, пятница. В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Объединим полученные результаты в одну таблицу:

In [18]:
# объявление функции number_tracks()с двумя параметрами: day, city
def number_tracks(day, city):
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

# Функция для подсчёта прослушиваний для конкретного города и дня.
# С помощью последовательной фильтрации с логической индексацией она 
# сначала получит из исходной таблицы строки с нужным днём,
# затем из результата отфильтрует строки с нужным городом,
# методом count() посчитает количество значений в колонке user_id. 
# Это количество функция вернёт в качестве результата

In [19]:
# dict-comprehension
pd.DataFrame({city : {day : number_tracks(day, city) for day in ['Monday','Wednesday','Friday']} for city in ['Moscow','Saint-Petersburg']})

,Moscow,Saint-Petersburg
Monday,15740,5614
Wednesday,11056,7003
Friday,15945,5895


#### 3.1.1 Выводы

Данные подтверждают разницу в поведении пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Санкт-Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Делаем вывод, что данные говорят в пользу первой гипотезы.

### 3.2 Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник, а также вечером в пятнцу в Москве преобладают одни жанры, а в Петербурге другие.

In [20]:
# информация по жителям Москвы
moscow_general = df[df['city'] == 'Moscow']

In [21]:
# информация по жителям СПб
spb_general = df[df['city'] == 'Saint-Petersburg']

Создадим функцию, которая вернет информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [39]:
# Объявление функции genre_weekday() с параметрами table, day, time1, time2

def genre_weekday(table, day, time1, time2):
    # в переменную genre_df сохраняются те строки переданного датафрейма table, для
#    которых одновременно:
#    - значение в столбце day равно значению аргумента day
#    - значение в столбце time больше значения аргумента time1
#    - значение в столбце time меньше значения аргумента time2
    genre_df = table[table['day'] == day]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df = genre_df[genre_df['time'] < time2]
    
    genre_df_count = genre_df.groupby('genre')['artist'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending = False)
    return genre_df_sorted.head(10)

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [41]:
# Москва, пн, утро
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: artist, dtype: int64

In [42]:
# Петербург, пн, утро
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: artist, dtype: int64

In [43]:
# Москва, пт, вечер
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: artist, dtype: int64

In [44]:
# Петербург, пт, вечер
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: artist, dtype: int64

#### 3.2.1 Выводы

Если сравнить топ-10 жанров в понедельник утром, можно сделать следующие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика;

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### 3.3 Жанровые предпочтения в Москве и Петербурге

Третья гипотеза о том, что Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра:

In [38]:
moscow_genres = moscow_general.groupby('genre')['day'].count().sort_values(ascending = False)
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: day, dtype: int64

Повторим для Петербурга:

In [25]:
spb_genres = spb_general.groupby('genre')['day'].count().sort_values(ascending = False)
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: day, dtype: int64

#### 3.3.1 Выводы

Третья гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Проверено три гипотезы и установилено:

1. Первая гипотеза полностью подтвердилась. 

День недели по-разному влияет на активность пользователей в Москве и Петербурге. 


2. Вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:

* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.


3. Третья гипотеза не подтвердилась.

Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

**На практике исследования содержат проверки статистических гипотез.**
Из данных одного сервиса не всегда можно сделать вывод о всех жителях города.
Проверки статистических гипотез покажут, насколько они достоверны, исходя из имеющихся данных. 
С методами проверок гипотез вы ещё познакомитесь в следующих темах.